In [ ]:
import tensorflow as tf
print("GPU",tf.config.list_physical_devices("GPU"))
print(tf.test.is_gpu_available())
print('valid tensorflow version')

首先对Tensorflow-gpu版本进行验证，若输出为True以及valid tensorflow version,代表gpu版本安装正确。

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential  #Sequential（翻译为堆叠、顺序即可）可以用来构建复杂的神经网络；
from keras.layers import Dense,Dropout,Activation,LSTM,Flatten,TimeDistributed,RepeatVector
from keras.layers.normalization import batch_normalization  #bach_normalizaiton定义一个标准化层
from keras.optimizers import Adam  #优化器选择Adam算法  Adam:adaptive learning rate optimization algorithm
from keras.callbacks import EarlyStopping,ModelCheckpoint  #Eerlystopping:当被监测的数量不再提升，则停止训练;  ModelCheckpoint:在每个训练期之后保存模型；
from sklearn.preprocessing import StandardScaler,MinMaxScaler  #数据预处理
import matplotlib.pyplot as plt
%matplotlib inline  


%matplotlib inline的功能是可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步。
该Step的作用是对导入LSTM模型所需要的库；

In [ ]:
n_in = 168  #168表示过去的七天的数据
n_out = 24  #24表示要预测一天
n_features = 1  #特征数为1
n_val = 1  #验整的数据量
n_epoachs = 250  #训练世代次数为250

数据的颗粒度是小时，时长是一个月，n_out = 24,预测一天的流量，根据经验选择n_in = 168选择七天的数据进行预测即可；

In [52]:
#d导入数据
def load_stw_data() -> pd.DataFrame:
    
    df_stw = pd.read_excel('data3.xlsx')
    df_stw.columns = ['BillingDate', 'VolumnHL']
    
    return df_stw

In [ ]:
#MinMaxScaler数据归一化，可以帮助网络模型更快的拟合，稍微有一些提高准确率的效果
def minmaxscaler(data: pd.DataFrame) -> pd.DataFrame:
    
    volume = data.VolumnHL.values
    volume = volume.reshape(len(volume), 1)
    volume = scaler.fit_transform(volume)  #这里fit_transform函数的输入比较严格，所以就需要做reshape，单纯的list是不行的
    volume = volume.reshape(len(volume),)    
    data['VolumnHL'] = volume
    return data


In [ ]:
#划分训练数据集和训练数据集，这里需要注意的是我们需要预测的数据是不可以出现在训练中的！
def split_data(x,y,n_test:int):
    x_train = x[:-n_val-n_out+1]
    x_val = x[-n_val:]
    y_train = y[:-n_val-n_out+1]
    y_val = y[-n_val:]
    return x_train, y_train, x_val, y_val


In [ ]:
#划分X和Y
def build_train(train, n_in, n_out):
    
    train = train.drop(["BillingDate"], axis=1)
    X_train, Y_train = [], []
    for i in range(train.shape[0]-n_in-n_out+1):
        X_train.append(np.array(train.iloc[i:i+n_in]))
        Y_train.append(np.array(train.iloc[i+n_in:i+n_in+n_out]["VolumnHL"]))
        
    return np.array(X_train), np.array(Y_train)
   

In [ ]:
#构建最简单的LSTM
def build_lstm(n_in: int, n_features: int):
    
    model = Sequential()
    model.add(LSTM(12, activation='relu', input_shape=(n_in, n_features)))
    model.add(Dropout(0.3))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mae')
    
    return model